# Recolectando datos de la API de Twitter.

Recordemos que las API o interfaces de programación de aplicaciónes , son métodos para acceder a los datos de una empresa u organización gubernamental.Muchas sino la mayoria de empresas de redes sociales tiene algun tipo de API que estan disponibles para desarrolladores e investigadores externos.

Twitter tiene múktiples API que se pueden usar para diferentes propósitos entre las cuales estan:

* **API de Busqueda** - Permite el acceso a los tweets desde hace una semana.

* **API de Anuncios** - Se centra en los anuncios de Twitter.

* **API de transmisión** - Permite recopilar una muestra de tweets en tiempo real en función de palabras clave, ID de usuarios y ubicaciones.

La conexión a esta API permanece abierta hasta que la cerramos.La API de transmición tiene dos puntos finales:

* **Filter endpoint** - Con esto podemos solicitar datos en algunas de cientas de palabras clave, algunos miles de usuario y 25 rangos de ubicación.



* **Sample endpoint**- Este punto , sirve par que Twitter pueda devolver una muestra del 1% de todo Twitter.

Para recopilar datos de la API de Twitter vamos a utilizar un paquete llamado `tweepy`, este paquete abstrae gran parte del trabajo que necesitamos para configurar una conexión estable de la API de transmisión o Streaming API de Twitter.

Cuando hagamos esto en la practica cada persona que lea esta información , tendra que configurar su propia cuenta de Twitter developer y sus propias claves para la autentucación.

`tweepy` requiere un objeto llamado `SListener` el cual nos dira cómo manejar los datos entrantes a continuación les dejo el código:




In [14]:
from tweepy.streaming import StreamListener
import time


class SListener(StreamListener):
    def __init__(self, api = None, fprefix = 'streamer'):
        self.api = api or API()
        self.counter = 0
        self.fprefix = fprefix
        self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')


    def on_data(self, data):
        if  'in_reply_to_status' in data:
            self.on_status(data)
        elif 'delete' in data:
            delete = json.loads(data)['delete']['status']
            if self.on_delete(delete['id'], delete['user_id']) is False:
                return False
        elif 'limit' in data:
            if self.on_limit(json.loads(data)['limit']['track']) is False:
                return False
        elif 'warning' in data:
            warning = json.loads(data)['warnings']
            print("WARNING: %s" % warning['message'])
            return


    def on_status(self, status):
        self.output.write(status)
        self.counter += 1
        if self.counter >= 20000:
            self.output.close()
            self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')
            self.counter = 0
        return


    def on_delete(self, status_id, user_id):
        print("Delete notice")
        return


    def on_limit(self, track):
        print("WARNING: Limitation notice received, tweets missed: %d" % track)
        return


    def on_error(self, status_code):
        print('Encountered error with status code:', status_code)
        return 


    def on_timeout(self):
        print("Timeout, sleeping for 60 seconds...")
        time.sleep(60)
        return 

Ahora hay que darnos una idea de la función de este constructor.

Nuestro nuevo objeto `SListener`hereda una clase general `StreamListener` incluida en el paquete `tweepy`, despues abre  un nuevo archivo con marca ded tiempo en el que almacena los tweets y toma un argumento API opcional.

Primero debemos autenticarnos con Twitter, este proceso requere de cuatro tokens que obtenemos del sitios de desarroladores de Twitter los cuales son :

* consumer key.
* consumer secret.
* token acces.
* acces token secret.

Primero importamos las heramientas a utilizar:

In [2]:
from tweepy import OAuthHandler
from tweepy import API



Despues asignamos a la variable `auth` nuestra clave de consumido y secreto de consumidor.

In [5]:

auth = OAuthHandler(consumer_key='',consumer_secret='')

Luego establecemos el token de acceso y el token de acceso secreto:

In [8]:

access_token = ''
access_token_secret = ''
auth.set_access_token(access_token,access_token_secret)

Finalmente pasamos el objeto de autenticacion al objeto API de tweepy.

In [9]:
api = API(auth)

Ahora podemos empezar a recopilar datos, tomaremos una muestra aleatoria de todo Twitter , por lo que primero instanciamos un objeto `SListener`.

In [10]:
from tweepy import Stream
listen = SListener(api)


Luego instanciamos un objeo `Stream`

In [12]:
stream = Stream(auth,listen)

Por ultimo llamamos la metodo `sample` para comenzar a recopilar datos.

In [13]:
stream.sample()

# Recopilando datos sobre Keywords.

Ahora que hemos configurado la autenticación, podemos comenzar a recopilar datos de Twitter. Recuerda que con la API de transmisión, recopilaremos datos de Twitter en tiempo real basados ​​en una muestra o filtrados por una palabra clave.

En nuestro ejemplo, recopilaremos datos sobre cualquier tweet que mencione #COVID o #Russia en el texto del tweet, el nombre de usuario o la descripción del usuario con el punto final del filtro.

In [ ]:
# importamos Stream
from tweepy import Stream

# creamos una lsta que contiene las palabras clave
keywords_to_track = ['#rstats']

# Pasamos los objetos de autenticacion y de stream.

listen = SListener(api)
stream = Stream(auth,listen)

# Coleccionamos los datos

stream.filter(track = keywords_to_track)